In [ ]:
# Data visualization
import numpy as np
import pandas as pd

# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
# from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
# import keras_tuner as kt

# Train-Test
from sklearn.model_selection import train_test_split
# Classification Report
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

import pickle

import warnings
warnings.filterwarnings('ignore')

## 1. Set up important landmarks and functions

In [ ]:
# Determine important landmarks for lunge
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
    "LEFT_HEEL",
    "RIGHT_HEEL",
    "LEFT_FOOT_INDEX",
    "RIGHT_FOOT_INDEX",
]

# Generate all columns of the data frame

HEADERS = ["label"] # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

TRAIN_SET_PATH  = "/content/err.train.csv"
TEST_SET_PATH  = "/content/err.test.csv"

In [ ]:
def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")

    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


# Remove duplicate rows (optional)
def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''

    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_dataset.csv", sep=',', encoding='utf-8', index=False)


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))

## 2. Describe and process data

In [ ]:
# load dataset
df = describe_dataset(TRAIN_SET_PATH)

# Categorizing label
df.loc[df["label"] == "L", "label"] = 0
df.loc[df["label"] == "C", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v', 'left_heel_x', 'left_heel_y', 'left_heel_z', 'left_heel_v', 'right_heel_x', 'right_heel_y', 'right_heel_z', 'right_heel_v', 'left_foot_index_x', 'left_foot_index_y', 'left_foot_index_z', 'left_foot_index_v', 'right_foot_index_x', 'right_foot_index_y', 'right_foot_index_z', 'right_foot_index_v']
Number of rows: 17907 
Number of columns: 53

Labels: 
label
L    9114
C    8793
Name: count, dtype: int64

Missing v

In [ ]:
from tensorflow.keras.utils import to_categorical
# Standard Scaling of features
with open("/content/input_scaler.pkl", "rb") as f2:
    input_scaler = pickle.load(f2)

x = df.drop("label", axis = 1)
x = pd.DataFrame(input_scaler.transform(x))

y = df["label"]

# # Converting prediction to categorical
y_cat = to_categorical(y)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2)

## 3. Train model

### 3.1. Set up

In [ ]:
stop_early = EarlyStopping(monitor='loss', patience=3)

# Final Results
final_models = {}

In [ ]:
def describe_model(model):
    '''
    Describe Model architecture
    '''
    print(f"Describe models architecture")
    for i, layer in enumerate(model.layers):
        number_of_units = layer.units if hasattr(layer, 'units') else 0

        if hasattr(layer, "activation"):
            print(f"Layer-{i + 1}: {number_of_units} units, func: ", layer.activation)
        else:
            print(f"Layer-{i + 1}: {number_of_units} units, func: None")


def get_best_model(tuner):
    '''
    Describe and return the best model found from keras tuner
    '''
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hps)

    describe_model(best_model)

    for h_param in ["learning_rate"]:
        print(f"{h_param}: {tuner.get_best_hyperparameters()[0].get(h_param)}")

    return best_model

### 3.2. Model with 3 layers

In [ ]:
def model_builder(hp):
    model = Sequential()
    model.add(Dense(50, input_dim = 52, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(2, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])

    return model

In [ ]:
# pip install keras-tuner


In [ ]:
import keras_tuner as kt
tuner = kt.Hyperband(
    model_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo'
)

In [ ]:
# tuner.search(x_train, y_train, epochs=10, callbacks=[stop_early])

In [ ]:
model = get_best_model(tuner)
model.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 52 units, func:  <function relu at 0x7990de7bf1c0>
Layer-2: 352 units, func:  <function tanh at 0x79907ff84040>
Layer-3: 2 units, func:  <function softmax at 0x79907ffb7be0>
learning_rate: 0.001
Epoch 1/100
1433/1433 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9673 - loss: 0.0834 - val_accuracy: 0.9994 - val_loss: 0.0030
Epoch 2/100
1433/1433 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9965 - loss: 0.0108 - val_accuracy: 0.9997 - val_loss: 0.0015
Epoch 3/100
1433/1433 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9993 - loss: 0.0022 - val_accuracy: 0.9997 - val_loss: 0.0016
Epoch 4/100
1433/1433 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9991 - loss: 0.0040 - val_accuracy: 0.9994 - val_loss: 0.0014
Epoch 5/100
1433/1433 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.9979 - loss: 0.0050 - val_accuracy: 0.9980 - val_loss: 0.0049
Epoch 6/100
1433/1433 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9997 - loss: 0.0014 - val_accuracy: 0.9994 

In [ ]:
final_models["3_layers"] = model

### 3.3. Model with 5 layers

In [ ]:
def model_builder_5(hp):
    model = Sequential()
    model.add(Dense(52, input_dim = 52, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])

    return model

In [ ]:
tuner = kt.Hyperband(
    model_builder_5,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_2'
)
# tuner.search(x_train, y_train, epochs=10, callbacks=[stop_early])

Reloading Tuner from keras_tuner_dir/keras_tuner_demo_2/tuner0.json


In [ ]:
model_5 = get_best_model(tuner)
model_5.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 52 units, func:  <function relu at 0x7990de7bf1c0>
Layer-2: 160 units, func:  <function relu at 0x7990de7bf1c0>
Layer-3: 288 units, func:  <function relu at 0x7990de7bf1c0>
Layer-4: 384 units, func:  <function relu at 0x7990de7bf1c0>
Layer-5: 2 units, func:  <function softmax at 0x79907ffb7be0>
learning_rate: 0.0001
Epoch 1/100
1433/1433 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9519 - loss: 0.1770 - val_accuracy: 0.9986 - val_loss: 0.0072
Epoch 2/100
1433/1433 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9979 - loss: 0.0073 - val_accuracy: 0.9992 - val_loss: 0.0050
Epoch 3/100
1433/1433 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9993 - loss: 0.0023 - val_accuracy: 0.9994 - val_loss: 0.0025


In [ ]:
final_models["5_layers"] = model_5

### 3.4. Model with 7 layers along with Dropout layers

In [ ]:
def model_builder_dropout_5(hp):
    model = Sequential()
    model.add(Dense(52, input_dim = 52, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dropout(0.5))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dropout(0.5))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])

    return model

In [ ]:
tuner = kt.Hyperband(
    model_builder_dropout_5,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_3'
)
# tuner.search(x_train, y_train, epochs=10, callbacks=[stop_early])

In [ ]:
model_5_with_dropout = get_best_model(tuner)
model_5_with_dropout.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 52 units, func:  <function relu at 0x7990de7bf1c0>
Layer-2: 224 units, func:  <function tanh at 0x79907ff84040>
Layer-3: 0 units, func: None
Layer-4: 288 units, func:  <function tanh at 0x79907ff84040>
Layer-5: 0 units, func: None
Layer-6: 128 units, func:  <function tanh at 0x79907ff84040>
Layer-7: 2 units, func:  <function softmax at 0x79907ffb7be0>
learning_rate: 0.01
Epoch 1/100
1433/1433 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9227 - loss: 0.2356 - val_accuracy: 0.9978 - val_loss: 0.0082
Epoch 2/100
1433/1433 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9930 - loss: 0.0291 - val_accuracy: 0.9983 - val_loss: 0.0101
Epoch 3/100
1433/1433 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9946 - loss: 0.0233 - val_accuracy: 0.9886 - val_loss: 0.0444


In [ ]:
final_models["7_layers_with_dropout"] = model_5_with_dropout

### 3.5. Model with 7 layers

In [ ]:
def model_builder_7(hp):
    model = Sequential()
    model.add(Dense(52, input_dim = 52, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_layer_4 = hp.Int('layer_4', min_value=32, max_value=512, step=32)
    hp_layer_5 = hp.Int('layer_5', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(units=hp_layer_4, activation=hp_activation))
    model.add(Dense(units=hp_layer_5, activation=hp_activation))
    model.add(Dense(2, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])

    return model

In [ ]:
pip install keras-tuner

In [ ]:
import keras_tuner as kt
tuner = kt.Hyperband(
    model_builder_7,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_4'
)
tuner.search(x_train, y_train, epochs=10, callbacks=[stop_early])

In [ ]:
model_7 = get_best_model(tuner)
model_7.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 52 units, func:  <function relu at 0x784721864670>
Layer-2: 288 units, func:  <function tanh at 0x7847211e3250>
Layer-3: 256 units, func:  <function tanh at 0x7847211e3250>
Layer-4: 192 units, func:  <function tanh at 0x7847211e3250>
Layer-5: 96 units, func:  <function tanh at 0x7847211e3250>
Layer-6: 480 units, func:  <function tanh at 0x7847211e3250>
Layer-7: 2 units, func:  <function softmax at 0x7847211e2e60>
learning_rate: 0.0001
Epoch 1/100
1433/1433 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9552 - loss: 0.1162 - val_accuracy: 0.9992 - val_loss: 0.0034
Epoch 2/100
1433/1433 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.9966 - loss: 0.0098 - val_accuracy: 0.9992 - val_loss: 0.0018
Epoch 3/100
1433/1433 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.9988 - loss: 0.0041 - val_accuracy: 0.9992 - val_loss: 0.0016
Epoch 4/100
1433/1433 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9985 - loss: 0.0040 - val_accuracy: 0.9983 - val_loss: 0.00

In [ ]:
model_7.save("/content/my_model.h5")

In [ ]:
final_models["7_layers"] = model_7

### 3.6. Final Models Description

In [ ]:
for name, model in final_models.items():
    print(f"{name}: ", end="")
    describe_model(model)
    print()

7_layers_with_dropout: Describe models architecture
Layer-1: 52 units, func:  <function relu at 0x15db71b80>
Layer-2: 288 units, func:  <function relu at 0x15db71b80>
Layer-3: 0 units, func: None
Layer-4: 224 units, func:  <function relu at 0x15db71b80>
Layer-5: 0 units, func: None
Layer-6: 320 units, func:  <function relu at 0x15db71b80>
Layer-7: 2 units, func:  <function softmax at 0x15db71160>

5_layers: Describe models architecture
Layer-1: 52 units, func:  <function relu at 0x15db71b80>
Layer-2: 480 units, func:  <function tanh at 0x15db71ee0>
Layer-3: 480 units, func:  <function tanh at 0x15db71ee0>
Layer-4: 192 units, func:  <function tanh at 0x15db71ee0>
Layer-5: 2 units, func:  <function softmax at 0x15db71160>

3_layers: Describe models architecture
Layer-1: 52 units, func:  <function relu at 0x15db71b80>
Layer-2: 192 units, func:  <function relu at 0x15db71b80>
Layer-3: 2 units, func:  <function softmax at 0x15db71160>

7_layers: Describe models architecture
Layer-1: 52 unit

## 4. Model Evaluation

### 4.1. Train set

In [ ]:
train_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(x_test, verbose=False)
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(y_test, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1])

    train_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

train_set_results.sort(key=lambda k: sum(k[3]), reverse=True)
pd.DataFrame(train_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,7_layers_with_dropout,"[1.0, 0.999]","[0.999, 1.0]","[1.0, 1.0]","[[1805, 1], [0, 1776]]"
1,3_layers,"[1.0, 0.999]","[0.999, 1.0]","[1.0, 1.0]","[[1805, 1], [0, 1776]]"
2,7_layers,"[1.0, 1.0]","[1.0, 1.0]","[1.0, 1.0]","[[1806, 0], [0, 1776]]"
3,5_layers,"[1.0, 0.994]","[0.994, 1.0]","[0.997, 0.997]","[[1795, 11], [0, 1776]]"


### 4.2. Test set evaluation

In [ ]:
test_df = pd.read_csv(TEST_SET_PATH)

# Categorizing label
test_df.loc[test_df["label"] == "L", "label"] = 0
test_df.loc[test_df["label"] == "C", "label"] = 1

In [ ]:
# Standard Scaling of features
test_x = test_df.drop("label", axis = 1)
test_x = pd.DataFrame(input_scaler.transform(test_x))

test_y = test_df["label"]

# # Converting prediction to categorical
test_y_cat = to_categorical(test_y)

In [ ]:
test_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(test_x, verbose=False)
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(test_y_cat, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1])

    test_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

test_set_results.sort(key=lambda k: sum(k[3]), reverse=True)
pd.DataFrame(test_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

2022-11-25 15:46:42.537983: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-25 15:46:42.694947: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-25 15:46:42.853052: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-25 15:46:42.974234: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,3_layers,"[0.998, 0.873]","[0.859, 0.998]","[0.923, 0.932]","[[482, 79], [1, 545]]"
1,7_layers_with_dropout,"[0.995, 0.786]","[0.736, 0.996]","[0.846, 0.879]","[[413, 148], [2, 544]]"
2,5_layers,"[0.963, 0.755]","[0.693, 0.973]","[0.806, 0.85]","[[389, 172], [15, 531]]"
3,7_layers,"[0.984, 0.687]","[0.561, 0.991]","[0.715, 0.812]","[[315, 246], [5, 541]]"


## 5. Dump Model

In [ ]:
# Dump the best model to a pickle file
with open("./model/dp/err_lunge_dp.pkl", "wb") as f:
    pickle.dump(final_models["3_layers"], f)

INFO:tensorflow:Assets written to: ram://4444bb6e-de2c-4bed-806c-db9d955522ad/assets


In [ ]:
with open("./model/dp/all_models.pkl", "wb") as f:
    pickle.dump(final_models, f)

INFO:tensorflow:Assets written to: ram://0f5761b9-5d62-4bf6-8de5-34355293f17a/assets
INFO:tensorflow:Assets written to: ram://ac2add31-fb1f-4baa-b925-718896fea315/assets
INFO:tensorflow:Assets written to: ram://20b6d7c9-7ddf-42a1-aaba-c31e12762a10/assets
INFO:tensorflow:Assets written to: ram://ee8c66ec-0ee3-4a5c-bc03-f12225018c47/assets


# Deployment

In [ ]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
import streamlit as st
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.models import load_model

# Load the trained model from the current directory
model = load_model("my_model.h5")

# Load scaler from the current directory
with open("input_scaler.pkl", "rb") as f:
    input_scaler = pickle.load(f)

# Define a function to make predictions on a DataFrame
def make_prediction(df):
    # Scale the input data
    scaled_data = input_scaler.transform(df)
    # Make a prediction
    predictions = model.predict(scaled_data)
    predicted_classes = np.argmax(predictions, axis=1)
    return predicted_classes

# Streamlit UI
st.title("ML Model Deployment with Streamlit")
st.write("Upload your CSV file to get predictions")

# File uploader for CSV
uploaded_file = st.file_uploader("Choose a CSV file", type="csv")

if uploaded_file is not None:
    # Read the CSV file
    test_data = pd.read_csv(uploaded_file)

    # Display the uploaded CSV file
    st.write("Uploaded CSV Data:")
    st.write(test_data.head())

    # Assuming your model expects the same features as the training data
    features = test_data.drop("label", axis=1, errors="ignore")  # Adjust if 'label' exists

    # Button to make predictions on the uploaded data
    if st.button("Make Prediction"):
        predictions = make_prediction(features)
        test_data["Predictions"] = predictions

        # Show the predictions
        st.write("Predictions:")
        st.write(test_data)

        # Option to download the predictions as CSV
        csv = test_data.to_csv(index=False).encode('utf-8')
        st.download_button(
            "Download Predictions as CSV",
            csv,
            "predictions.csv",
            "text/csv",
            key="download-csv"
        )


2024-09-11 02:03:08.700 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-11 02:03:08.813 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-09-11 02:03:08.817 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-11 02:03:08.820 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-11 02:03:08.824 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-11 02:03:08.827 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-11 02:03:08.830 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-11 02:03:08.832 Thread 'MainThread': mi